In [8]:
%reload_ext autoreload
%autoreload 3

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import matplotlib.animation as animation
from scipy.stats import binom, t
import matplotlib as mpl
from sklearn import datasets
import itertools
from tqdm import tqdm

from utils import *

In [9]:
X_0 = np.random.rand(50,2) + 1
X_1 = np.random.rand(50,2) - 1
X = np.concatenate([X_0, X_1])

x_min, x_max = min(X[:,0]), max(X[:,0])
y_min, y_max = min(X[:,1]), max(X[:,1])

In [10]:
def build_rand_mat(K, x_min, y_min, x_max, y_max):
    X_H0 = np.random.uniform(size=(K,2)) * np.array([x_max - x_min, y_max - y_min]) + np.array([x_min, y_min])
    W, D_sqinv = compute_G0(X_H0)
    L = compute_laplacian(W, D_sqinv)
    A = compute_AM_normalised(L, D_sqinv, 3) #3 peut être faible
    W, g = compute_Ge(A)
    g.compute_A()
    g.kruskal_algo()
    mat = g.compute_mat()
    return mat, g

In [11]:
def hash_function(V_, K, N):
    return int((V_ - 1) * K / N)

In [32]:
def update_deltas(deltas, eps, K, size, x_min, y_min, x_max, y_max, update_, Q=30, conf_level=0.05):
    res_q = np.zeros((K, Q), dtype=int)
    for q in range(Q):
        res = np.zeros(K, dtype=int)
        mat, g = build_rand_mat(size - 1, x_min, y_min, x_max, y_max)
        omega = g.omega
        V = g.V
        for i in range(len(mat)):
            k = hash_function(int(mat[i, 5]), K, size) 
            pfa = compute_pfa(min(mat[i, 4] / omega, 1), V , mat[i,5])
            if pfa < deltas[k]:
                res[k] += 1
        res_q[:, q] = res.copy()
    s = np.maximum(np.std(res_q, axis=1), 1e-3)
    m = np.mean(res_q, axis=1)
    upd = t.cdf((m - eps / K) / s * np.sqrt(Q - 1), Q-1) > conf_level
    deltas -= (2 * upd - 1) * update_
    deltas = np.maximum(0, np.minimum(1, deltas))
    return deltas, res_q


In [35]:
def compute_deltas(size, Q=30, conf_level=0.05, eps=1, K=10):
    deltas = 0.01 * np.ones(K)
    update_ = 1e-3
    for i in tqdm(range(100)):
        if i == 10:
            update_ = 1e-4
        if i == 20:
            update_ = 1e-5
        if i == 30:
            update_ = 1e-6
        if i == 50:
            update_ = 1e-7
        if i == 60:
            update_ = 1e-8
        if i == 70:
            update_ = 1e-9
        if i == 80:
            update_ = 1e-10
        if i == 90:
            update_ = 1e-11
        deltas, res_q = update_deltas(deltas, eps, K, size, x_min, y_min, x_max, y_max, update_, Q, conf_level)
        #print(np.mean(np.sum(res_q, axis=0)))
    return deltas

In [34]:
compute_deltas(50, Q=30, conf_level=0.05, eps=1, K=10)

100%|██████████| 100/100 [14:32<00:00,  8.73s/it]


array([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 1.8000e-09, 1.5180e-07, 6.3574e-06, 0.0000e+00])

In [21]:
K = 10
size = 30
deltas = (1 - 1e-3) * np.ones(K)
deltas = 0.01 * np.ones(K)
eps = 1
conf_level = 0.01
update_ = 1e-3
Q = 50

prev_deltas = deltas.copy()

for i in range(501):
    if i == 10:
        update_ = 1e-4
    if i == 20:
        update_ = 1e-5
    if i == 30:
        update_ = 1e-6
    if i == 60:
        update_ = 1e-7
    deltas, res_q = update_deltas(deltas, eps, K, size, x_min, y_min, x_max, y_max, update_, Q=Q)
    if i % 5 == 0:
        diff_delta = np.mean(np.abs(prev_deltas - deltas))
        prev_deltas = deltas.copy()
        print(i, np.mean(res_q), np.mean(np.sum(res_q, axis=0)), diff_delta)
        print(np.mean(res_q, axis=1))
        print(deltas)
print(deltas)

0 5.652 56.52 0.0009999999999999992
[42.92  6.4   2.76  1.18  0.78  0.62  0.36  0.44  0.06  1.  ]
[0.009 0.009 0.009 0.009 0.009 0.009 0.009 0.009 0.009 0.009]
5 5.656 56.56 0.004600000000000002
[43.28  6.24  2.76  1.3   0.78  0.52  0.46  0.16  0.06  1.  ]
[0.004 0.004 0.004 0.004 0.004 0.004 0.004 0.004 0.008 0.004]
10 0.012 0.12 0.003520000000000001
[0.   0.   0.   0.   0.   0.   0.   0.06 0.06 0.  ]
[0.0001 0.0001 0.0001 0.0001 0.0001 0.0001 0.0001 0.0019 0.0099 0.0001]
15 5.55 55.5 0.00011999999999999984
[4.298e+01 6.620e+00 2.620e+00 1.140e+00 7.200e-01 2.600e-01 6.000e-02
 2.000e-02 8.000e-02 1.000e+00]
[0.     0.     0.     0.     0.     0.     0.0002 0.0018 0.0096 0.    ]
20 0.008 0.08 8.59999999999999e-05
[0.   0.   0.   0.   0.   0.   0.06 0.02 0.   0.  ]
[1.00e-05 1.00e-05 1.00e-05 1.00e-05 1.00e-05 1.00e-05 3.90e-04 2.21e-03
 9.41e-03 1.00e-05]
25 5.392 53.92 1.1999999999999965e-05
[4.306e+01 6.560e+00 2.480e+00 6.400e-01 4.000e-02 8.000e-02 0.000e+00
 4.000e-02 2.000e-02 1

KeyboardInterrupt: 

In [56]:
res = np.zeros(K, dtype=int)
mat, g = build_rand_mat(size - 1, x_min, y_min, x_max, y_max)
omega = g.omega
V = g.V
for i in range(len(mat)):
    k = hash_function(int(mat[i, 5]), K, size) 
    pfa = compute_pfa(mat[i, 4] / omega, V , mat[i,5])
    print(pfa)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999911384875
0.9999999798208312
0.9999999700691523
0.9999999875998891
0.9999999047643218
0.9999998613794215
0.9999998451084481
0.9999999604144829
0.999999981745761
0.9999999021206116
0.9999998940045819
0.9999999082609087
0.9999999616894677
0.9999999688822109
0.9999991706630476
0.999999962033505
0.9999987211584747
0.9999999106375322
0.9999986363246771
0.9999997891130711
0.9999993742809271
0.9999998971524051
0.9999989742958838
0.9999992421055128
0.9999842277884835
0.9999939208290389
0.9998045073352191
nan


In [ ]:
K = 10
deltas = (1 - 1e-3) * np.ones(K)
deltas = 0.98 * np.ones(K)
eps = 1
conf_level = 0.01
update_ = 1e-4
Q = 50

prev_deltas = deltas.copy()

for i in range(5001):
    if i == 500:
        update_ = 1e-5
    deltas, res_q = update_deltas(deltas, eps, mat, K, x_min, y_min, x_max, y_max, update_, Q=Q)
    if i % 50 == 0:
        diff_delta = np.mean(np.abs(prev_deltas - deltas))
        prev_deltas = deltas.copy()
        print(i, np.mean(res_q), np.mean(np.sum(res_q, axis=0)), diff_delta)
    if i % 200 == 0:
        print(deltas)
print(deltas)

In [21]:
K = 20
deltas = (1 - 1e-2) * np.ones(K)
eps = 0.1
conf_level = 0.05
update_ = 1e-5
Q = 50

for i in range(1001):
    deltas, res_q = update_deltas(deltas, eps, mat, K, x_min, y_min, x_max, y_max, update_, Q=Q)
    if i % 50 == 0:
        print(i, np.mean(res_q), np.mean(np.sum(res_q, axis=0)))
print(deltas)

0 0.037 0.74
50 0.031 0.62
100 0.036 0.72
150 0.032 0.64
200 0.03 0.6
250 0.025 0.5
300 0.023 0.46
350 0.025 0.5
400 0.033 0.66
450 0.018 0.36
500 0.025 0.5
550 0.027 0.54


KeyboardInterrupt: 

In [33]:
((1 - 1e-2) - deltas[9]) / update_

-1000.0000000000008

In [48]:
deltas[2]

0.9999699999999979

In [16]:
np.sum(res_q, axis=0)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 2, 0, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 4])

In [11]:
s = np.maximum(np.std(res_q, axis=1), 1e-5)
m = np.mean(res_q, axis=1)
t.cdf((m - eps / K) / s * np.sqrt(Q - 1), Q-1) > conf_level

array([False, False,  True,  True, False,  True,  True,  True,  True,
       False])

In [25]:
deltas

array([0.98534, 0.98534, 0.98534, 0.98534, 0.9854 , 0.99328, 0.99466,
       0.99466, 0.99466, 0.98534])